In [55]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sklearn
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import r2_score
from math import sqrt

In [8]:
df = pd.read_csv("cleaned2.csv")

In [9]:
df.head()

,Unnamed: 0,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,state
0,0,35990,2010.0,chevrolet,good,8 cylinders,gas,32742.0,clean,other,2,other,south
1,1,7500,2014.0,hyundai,excellent,4 cylinders,gas,93600.0,clean,automatic,2,sedan,south
2,2,4900,2006.0,bmw,good,6 cylinders,gas,87046.0,clean,automatic,2,SUV,south
3,5,29590,2016.0,toyota,good,6 cylinders,gas,33290.0,clean,other,4,pickup,south
4,6,39990,2012.0,ford,good,8 cylinders,gas,9692.0,clean,other,2,coupe,south


In [10]:
df["price"]=np.log1p(df["price"])

## Encoding

### For Numerical Data

#### year

In [11]:
year_min = df["year"].min()
year_max = df["year"].max()
df["year"] = ((df["year"] - year_min) / (year_max - year_min))

#### odometer

In [12]:
odo_min = df["odometer"].min()
odo_max = df["odometer"].max()
df["odometer"] = ((df["odometer"] - odo_min) / (odo_max - odo_min))

### For Ordinal Data

#### cylinders

In [13]:
label_encoder = LabelEncoder()
label_encoder.fit_transform(list(df['cylinders'].astype(str).values))
df['cylinders'] = label_encoder.transform(list(df['cylinders'].astype(str).values))

In [14]:
#Normalizing Cylinder
cyl_min = df["cylinders"].min()
cyl_max = df["cylinders"].max()
df["cylinders"] = ((df["cylinders"] - cyl_min) / (cyl_max - cyl_min))

### OneHotEncoding

In [15]:
train_df = pd.get_dummies(df)

In [16]:
len(train_df)

318824

In [17]:
train_df.head()

,Unnamed: 0,price,year,cylinders,odometer,drive,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,...,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,state_midwest,state_northeast,state_south,state_west
0,0,10.491024,0.583333,0.857143,0.130950,2,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,8.922792,0.750000,0.428571,0.374431,2,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,2,8.497195,0.416667,0.714286,0.348210,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,5,10.295226,0.833333,0.714286,0.133143,4,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,6,10.596410,0.666667,0.857143,0.038732,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### Splitting the Data into test and train

In [18]:
y = train_df["price"]
x = train_df.drop(['price', 'Unnamed: 0'], axis=1)
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(x,y,test_size=0.2,random_state=42)

In [19]:
x_train.head()

,year,cylinders,odometer,drive,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,...,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,state_midwest,state_northeast,state_south,state_west
220052,0.958333,0.714286,0.040324,2,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
10665,0.333333,0.428571,0.273475,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
102772,0.750000,0.428571,0.308018,2,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
90016,0.791667,0.428571,0.528394,4,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
294161,0.708333,1.000000,0.386033,4,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


### GradientBoosting Model

In [48]:
#GradientBoosting model
params = {'n_estimators': 500, 'max_depth': 8,'learning_rate': 0.1}
gb_model = ensemble.GradientBoostingRegressor(**params)

In [49]:
#Training the Gradient boost model
gb_model = gb_model.fit(x_train, y_train)

In [50]:
#Predict test
y_pred=gb_model.predict(x_test)
#Predict train
train_pred = gb.model(x_train)

In [58]:
#train 
mse1 = mean_squared_error(y_train, train_pred)
rmse1 = sqrt(mse1)
print("rmse(Train):", rmse1)
#test
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
print("rmse(Test):", rmse)

rmse(Train): 0.2754892285733874
rmse(Test): 0.250810743490817


In [59]:
#train
r_sqr1 = r2_score(y_train, train_pred)
print("r_squared(Train):",r_sqr1)
#test
r_sqr = r2_score(y_test, y_pred)
print("r_squared(Test):",r_sqr)

r_squared(Train): 0.8577193686904432
r_squared(Test): 0.8813092063873208


In [101]:
#Feature Importance Gradient Boosting
feature_imp = pd.DataFrame(gb_model.feature_importances_,index=x_train.columns)
feature_imp["columns"] = feature_imp.index
feature_imp.index=range(0,len(feature_imp))
feature_imp.columns=["score","features"]
feature_imp

,score,features
0,0.494727,year
1,0.092697,cylinders
2,0.142817,odometer
3,0.059916,drive
4,0.000632,manufacturer_acura
...,...,...
73,0.000638,type_wagon
74,0.001762,state_midwest
75,0.001423,state_northeast
76,0.001701,state_south


In [102]:
categorical_columns=['manufacturer','condition','cylinders','fuel','title_status','transmission','type','state']
for cat in categorical_columns:
    sco = (feature_imp[feature_imp["features"].str.startswith(cat)]["score"].sum())/len(feature_imp[feature_imp["features"].str.startswith(cat)]["score"])
    print(cat,sco)
    
for index,row in feature_imp.iterrows():
    if(row["features"]=="odometer"):
        print("Odometer",(row["score"]))
    if(row["features"]=="year"):
        print("Year",(row["score"]))

manufacturer 0.0015327452322229417
condition 0.0019328435485013833
cylinders 0.09269724245759933
fuel 0.010308950011526204
title_status 0.0018380760044626983
transmission 0.001890568326434982
type 0.004924620932434968
state 0.0026463041886821883
Year 0.49472698991168457
Odometer 0.14281664401045202


### XGBoost

In [64]:
#XGBoost Model
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=5
)

In [65]:
#Training the XGBoost Model
regressor.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [66]:
#Predict test
y_pred = regressor.predict(x_test)
#Predict train
train_pred = regressor.predict(x_train)

In [67]:
#train
mse_train = mean_squared_error(y_train, train_pred)
rmse = math.sqrt(mse_train)
print("rmse(Train):",rmse)
#test
mse = mean_squared_error(y_test,y_pred)
rmse=math.sqrt(mse)
rmse
print("rmse(Test):",rmse)

rmse(Train): 0.2754892285733874
rmse(Test): 0.28333842752516675


In [68]:
#train
r_sqr1 = r2_score(y_train, train_pred)
print("r_squared(Train):",r_sqr1)
#test
r_sqr = r2_score(y_test, y_pred)
print("r_squared(Test):",r_sqr)

r_squared(Train): 0.8577193686904432
r_squared(Test): 0.8485268276098412


In [103]:
#feature importance XGBoost
feature_imp = pd.DataFrame(regressor.feature_importances_,index=x_train.columns)
feature_imp["columns"] = feature_imp.index
feature_imp.index=range(0,len(feature_imp))
feature_imp.columns=["score","features"]
feature_imp

,score,features
0,0.114812,year
1,0.028663,cylinders
2,0.027604,odometer
3,0.045809,drive
4,0.004947,manufacturer_acura
...,...,...
73,0.003875,type_wagon
74,0.003166,state_midwest
75,0.004964,state_northeast
76,0.002108,state_south


In [104]:
categorical_columns=['manufacturer','condition','cylinders','fuel','title_status','transmission','type','state']
for cat in categorical_columns:
    sco = (feature_imp[feature_imp["features"].str.startswith(cat)]["score"].sum())/len(feature_imp[feature_imp["features"].str.startswith(cat)]["score"])
    print(cat,sco)
    
for index,row in feature_imp.iterrows():
    if(row["features"]=="odometer"):
        print("Odometer",(row["score"]))
    if(row["features"]=="year"):
        print("Year",(row["score"]))

manufacturer 0.009860224113231751
condition 0.00329400971531868
cylinders 0.02866264618933201
fuel 0.024207159876823425
title_status 0.0071942514429489774
transmission 0.006313098594546318
type 0.013758071340047397
state 0.005489964038133621
Year 0.11481241881847382
Odometer 0.027603738009929657
